In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
pokemon_df = pd.read_csv("pokemon.csv")
pokemon_df.head()

In [ ]:
pokemon_df = pokemon_df.set_index("#")
pokemon_df.head()

In [ ]:
combats_df = pd.read_csv("combats.csv")
combats_df.head()

In [ ]:
pokemon_df.info()

In [ ]:
pokemon_df["Type 2"].value_counts(dropna=False)

In [ ]:
pokemon_df["Type 2"].fillna("empty", inplace=True)
pokemon_df["Type 2"].value_counts()

In [ ]:
print(pokemon_df.dtypes)
print("-" * 30)
print(combats_df.dtypes)

In [ ]:
pokemon_df["Type 1"] = pokemon_df["Type 1"].astype("category")
pokemon_df["Type 2"] = pokemon_df["Type 2"].astype("category")
pokemon_df["Legendary"] = pokemon_df["Legendary"].astype("int")
print(pokemon_df.dtypes)

In [ ]:
df_type1_onehot = pd.get_dummies(pokemon_df["Type 1"])
df_type1_onehot.head()

In [ ]:
df_type2_onehot = pd.get_dummies(pokemon_df["Type 2"])
df_type2_onehot.head()

In [ ]:
combine_df_onehot = df_type1_onehot.add(df_type2_onehot, fill_value=0).astype("int")

pd.options.display.max_columns = 30
pokemon_df = pokemon_df.join(combine_df_onehot)
pokemon_df.head()

In [ ]:
dict(enumerate(pokemon_df["Type 1"].cat.categories))

In [ ]:
pokemon_df["Type 1"].cat.codes.head(10)

In [ ]:
pokemon_df["Type 1"] = pokemon_df["Type 1"].cat.codes
pokemon_df["Type 2"] = pokemon_df["Type 2"].cat.codes
pokemon_df.head()

In [ ]:
pokemon_df.drop("Name", axis=1, inplace=True)
pokemon_df

In [ ]:
combats_df["Winner"] = combats_df.apply(lambda x: 0 if x.Winner == x.First_pokemon else 1, axis=1)
combats_df.head()

In [ ]:
data_num = combats_df.shape[0]
indexes = np.random.permutation(data_num)

train_indexes = indexes[:int(data_num*0.6)]
val_indexes = indexes[int(data_num*0.6):int(data_num*0.8)]
test_indexes = indexes[int(data_num*0.6):]

train_data = combats_df.loc[train_indexes]
val_data = combats_df.loc[val_indexes]
test_data = combats_df.loc[test_indexes]

In [ ]:
pokemon_df["Type 1"] = pokemon_df["Type 1"] / 19
pokemon_df["Type 2"] = pokemon_df["Type 2"] / 19

mean = pokemon_df.loc[:, "HP":"Generation"].mean()
std = pokemon_df.loc[:, "HP":"Generation"].std()

print(mean)
print(std)

pokemon_df.loc[:, "HP":"Generation"] = (pokemon_df.loc[:, "HP":"Generation"] - mean) / std
pokemon_df.head()

In [ ]:
print(pokemon_df.loc[:, "HP":"Generation"].mean())
print(pokemon_df.loc[:, "HP":"Generation"].std())

In [ ]:
x_train_index = np.array(train_data.drop("Winner", axis=1))
x_val_index = np.array(val_data.drop("Winner", axis=1))
x_test_index = np.array(test_data.drop("Winner", axis=1))

print(x_train_index)

In [ ]:
y_train = np.array(train_data["Winner"])
y_val = np.array(val_data["Winner"])
y_test = np.array(test_data["Winner"])

print(y_train)

## 第一種：屬性以數值表示

In [ ]:
pokemon_data_normal = np.array(pokemon_df.loc[:, :"Legendary"])
print(pokemon_data_normal.shape)

x_train_normal = pokemon_data_normal[x_train_index-1].reshape((-1, 20))
x_val_normal = pokemon_data_normal[x_val_index-1].reshape((-1, 20))
x_test_normal = pokemon_data_normal[x_test_index-1].reshape((-1, 20))
print(x_train_normal.shape)

In [ ]:
inputs = keras.Input(shape=(20, ))
x = layers.Dense(64, activation="relu") (inputs)
x = layers.Dropout(0.3) (x)
x = layers.Dense(64, activation="relu") (x)
x = layers.Dropout(0.3) (x)
x = layers.Dense(16, activation="relu") (x)
x = layers.Dropout(0.3) (x)
outputs = layers.Dense(1, activation="sigmoid") (x)

model_1 = keras.Model(inputs, outputs, name="model-1")
model_1.summary()

In [ ]:
model_1.compile(keras.optimizers.Adam(), loss=keras.losses.BinaryCrossentropy(), metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
model_dir = "logs/models"
if not os.path.exists(model_dir):
    print("new dir!")
    os.makedirs(model_dir)

In [ ]:
log_dir = os.path.join("logs", "model-1")

model_cbk = keras.callbacks.TensorBoard(log_dir=log_dir)
model_mckp = keras.callbacks.ModelCheckpoint(model_dir+"/best-model-1.h5", monitor="val_binary_accuracy",
                                             save_best_only=True, mode="max")

In [ ]:
history_1 = model_1.fit(x_train_normal, y_train, batch_size=64, epochs=200,
                        validation_data=(x_val_normal, y_val), callbacks=[model_cbk, model_mckp])

## 第二種：屬性以one hot表示

In [ ]:
pokemon_data_onehot = np.array(pokemon_df.loc[:, "HP":])
print(pokemon_data_onehot.shape)

x_train_onehot = pokemon_data_onehot[x_train_index-1].reshape((-1, 54))
x_val_onehot = pokemon_data_onehot[x_val_index-1].reshape((-1, 54))
x_test_onehot = pokemon_data_onehot[x_test_index-1].reshape((-1, 54))
print(x_train_onehot.shape)

In [ ]:
inputs = keras.Input(shape=(54, ))
x = layers.Dense(64, activation="relu") (inputs)
x = layers.Dropout(0.3) (x)
x = layers.Dense(64, activation="relu") (x)
x = layers.Dropout(0.3) (x)
x = layers.Dense(16, activation="relu") (x)
x = layers.Dropout(0.3) (x)
outputs = layers.Dense(1, activation="sigmoid") (x)

model_2 = keras.Model(inputs, outputs, name="model-2")
model_2.summary()

In [ ]:
model_2.compile(keras.optimizers.Adam(), loss=keras.losses.BinaryCrossentropy(), metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
model_dir = "logs/models"
if not os.path.exists(model_dir):
    print("new dir!")
    os.makedirs(model_dir)

In [ ]:
log_dir = os.path.join("logs", "model-2")

model_cbk = keras.callbacks.TensorBoard(log_dir=log_dir)
model_mckp = keras.callbacks.ModelCheckpoint(model_dir+"/best-model-2.h5", monitor="val_binary_accuracy",
                                             save_best_only=True, mode="max")

In [ ]:
history_2 = model_2.fit(x_train_onehot, y_train, batch_size=64, epochs=200,
                        validation_data=(x_val_onehot, y_val), callbacks=[model_cbk, model_mckp])

## 比較兩種模型

In [ ]:
plt.plot(history_1.history["binary_accuracy"], label="m1-train")
plt.plot(history_1.history["val_binary_accuracy"], label="m1-val")
plt.plot(history_2.history["binary_accuracy"], label="m2-train")
plt.plot(history_2.history["val_binary_accuracy"], label="m2-train")
plt.title("Binary Accuracy")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.legend()

In [ ]:
model_1 = keras.models.load_model(model_dir+"/best-model-1.h5")
model_2 = keras.models.load_model(model_dir+"/best-model-2.h5")

scores_1 = model_1.evaluate(x_test_normal, y_test)
scores_2 = model_2.evaluate(x_test_onehot, y_test)
print(f"model-1 acc:", scores_1[1])
print(f"model-2 acc:", scores_2[1])

## 妙蛙花、噴火龍、水箭龜 大PK

In [ ]:
venusaur = np.expand_dims(pokemon_data_onehot[3], axis=0)   # 妙蛙花
charizard = np.expand_dims(pokemon_data_onehot[7], axis=0)  # 噴火龍
blastoise = np.expand_dims(pokemon_data_onehot[12], axis=0) # 水箭龜

pred = model_2.predict(np.concatenate([venusaur, charizard], axis=-1))
winner = "妙蛙花" if pred < 0.5 else "噴火龍"
print(f"pred={pred}, winner is {winner}!")

pred = model_2.predict(np.concatenate([charizard, blastoise], axis=-1))
winner = "噴火龍" if pred < 0.5 else "水箭龜"
print(f"pred={pred}, winner is {winner}!")

pred = model_2.predict(np.concatenate([venusaur, blastoise], axis=-1))
winner = "妙蛙花" if pred < 0.5 else "水箭龜"
print(f"pred={pred}, winner is {winner}!")

## 隨機抽樣兩隻寶可夢對戰

In [ ]:
pokemon_info_df = pd.read_csv("pokemon.csv")
pokemon_info_df

In [ ]:
two_pokemon = np.random.choice(np.arange(800), size=2, replace=True)
print(two_pokemon)

first_pokemon = two_pokemon[0]
second_pokemon = two_pokemon[1]

In [ ]:
first_pokemon_name = pokemon_info_df.iloc[first_pokemon]["Name"]
second_pokemon_name = pokemon_info_df.iloc[second_pokemon]["Name"]

print(f"{first_pokemon_name}({first_pokemon}) vs. {second_pokemon_name}({second_pokemon})")

first_pokemon_data = np.expand_dims(pokemon_data_onehot[first_pokemon], axis=0)   # 妙蛙花
second_pokemon_data = np.expand_dims(pokemon_data_onehot[second_pokemon], axis=0)  # 噴火龍

print(np.concatenate([first_pokemon_data, second_pokemon_data], axis=-1))

pred = model_2.predict(np.concatenate([first_pokemon_data, second_pokemon_data], axis=-1))
winner = first_pokemon_name if pred < 0.5 else second_pokemon_name
print("pred={:.6f}, winner is {}!".format(pred[0][0], winner))